In [1]:
from typing import Tuple

In [3]:
test_tuple = (1,2)
test_tuple

(1, 2)

In [4]:
test_tuple[0]= 9
test_tuple

TypeError: 'tuple' object does not support item assignment

In [6]:
from datetime import datetime

# Your original dictionary
original_dict = {
    datetime(2023, 9, 1): 10.5,
    datetime(2023, 9, 3): 15.2,
    datetime(2023, 9, 2): 12.7,
}

# Convert the dictionary to a sorted list of tuples
sorted_list = sorted(original_dict.items())

# Create a new dictionary from the sorted list
sorted_dict = dict(sorted_list)

# Now you can access elements using datetime keys

sorted_dict


{datetime.datetime(2023, 9, 1, 0, 0): 10.5,
 datetime.datetime(2023, 9, 2, 0, 0): 12.7,
 datetime.datetime(2023, 9, 3, 0, 0): 15.2}

In [8]:
list(sorted_dict.items())[0]

(datetime.datetime(2023, 9, 1, 0, 0), 10.5)

In [17]:
from datetime import datetime,timedelta
from_dt = datetime(2023, 9, 1)
untill_dt =  datetime(2023, 9, 11)
cc = 3
ts = untill_dt - from_dt
step_ts = timedelta(days=int(ts.days/cc))
cur_dt = from_dt
for i in range(cc-1):
  new_untill = cur_dt + step_ts
  if new_untill > untill_dt:
    new_untill = untill_dt
  print(f"{cur_dt}- {new_untill}")
  cur_dt = new_untill
print(f"{cur_dt}- {untill_dt}")

2023-09-01 00:00:00- 2023-09-04 00:00:00
2023-09-04 00:00:00- 2023-09-07 00:00:00
2023-09-07 00:00:00- 2023-09-11 00:00:00


In [1]:
import src.strategy.run_config as cfg

In [2]:
str_set = cfg.StrategyConfigSet.Builder().add_set("A", [1,2]).add_set(
          "B", [4]).build()
expected_record1 = cfg.StrategyConfig({"A":1, "B":4})
expected_record2 = cfg.StrategyConfig({"A":2, "B":4})

In [3]:
str_set.as_records()[0]

In [4]:
hash(str_set.as_records()[0])

8094212078855480490

In [11]:
import concurrent.futures

arr = [1,2,3]
arr_tpl = [(v,v*2) for v in arr]

result = []

def do(v_tpl):
  v1 = v_tpl[0]
  v2 = v_tpl[1]
  result.append(v1)
  print(f"v1:{v1}, v2:{v2}")


with concurrent.futures.ProcessPoolExecutor(max_workers=5) as executor:
  # Map the do function to the list of tasks
  executor.map(do, arr_tpl)
result

v1:2, v2:4v1:1, v2:2v1:3, v2:6




[]

In [19]:
import concurrent.futures

result = []

def process_rep(tpl):
    v1 = tpl[0]
    v2 = tpl[1]
    print(f"v1:{v1}, v2:{v2}\n")
    return v1 * v2

# Assuming rep_list is your list of tasks
rep_list =  [(v,v*2) for v in [1,2,3,4]]

with concurrent.futures.ProcessPoolExecutor() as executor:
    new_result = list(executor.map(process_rep, rep_list))
result.extend(new_result)
print(rep_list)
result

IndentationError: expected an indented block after class definition on line 4 (761719347.py, line 5)

In [1]:
from ast import arg
import concurrent.futures
from concurrent.futures import thread
import multiprocessing
import time
class Worker:
    def __init__(self,lock, lock2) -> None:
        self.last_v = -1
        self.lock = lock
        self.lock2 = lock2
        pass
    def do(self, v):      
      self.last_v = v
      with self.lock:
        with self.lock2:
          print(f"v:{v}\n")
          time.sleep(v)      
          print(f"v:{v} last_v:{self.last_v}\n")
          return v * self.last_v
      
m = multiprocessing.Manager()
lock = m.Lock()
lock2 = m.Lock()

input_arr = [1,2,3,4,5,6]
result = []
worker = Worker(lock,lock2)
with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
    new_result = list(executor.map(worker.do, input_arr))
result.extend(new_result)
result

v:1



v:1 last_v:1

v:2

v:2 last_v:2

v:3

v:3 last_v:3

v:4

v:4 last_v:4

v:5

v:5 last_v:5

v:6

v:6 last_v:6



[1, 4, 9, 16, 25, 36]

In [2]:
from ast import arg
import concurrent.futures
from concurrent.futures import thread
import multiprocessing
import time
class Worker:
    def __init__(self) -> None:
        self.last_v = -1
        m = multiprocessing.Manager()
        self.lock = m.Lock()
        pass
    def do(self, v):      
      self.last_v = v
      with self.lock:
        print(f"v:{v}\n")
        time.sleep(v)      
        print(f"v:{v} last_v:{self.last_v}\n")
        return v * self.last_v
      

input_arr = [1,2,3,4,5,6]
result = []
worker = Worker()
with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
    new_result = list(executor.map(worker.do, input_arr))
result.extend(new_result)
result

v:1

v:1 last_v:1

v:2

v:2 last_v:2

v:3

v:3 last_v:3

v:4

v:4 last_v:4

v:5

v:5 last_v:5

v:6

v:6 last_v:6



KeyboardInterrupt: 

In [2]:
from ast import arg
import concurrent.futures
from concurrent.futures import thread
import multiprocessing
import time
class Worker:
    def __init__(self) -> None:
        self.last_v = 0
        pass
    def do(self, args):
      v = args[0]
      lock = args[1]
      thread_id = multiprocessing.current_process().ident
      with lock:
        print(f"start:  \tthread_id:{thread_id }\tlast_v:{self.last_v}\tv:{v}\n")
        self.last_v = v + self.last_v
        print(f"changed:\tthread_id:{thread_id }\tlast_v:{self.last_v}\tv:{v}\n")
        time.sleep(v)      
        print(f"sleeped:\tthread_id:{thread_id }\tlast_v:{self.last_v}\tv:{v}\n")
        return v * self.last_v
      
m = multiprocessing.Manager()
lock = m.Lock()

input_arr = [(v,lock) for v in [1,2,3,4,5,6]]
result = []
worker = Worker()
with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
    new_result = list(executor.map(worker.do, input_arr))
result.extend(new_result)
result

start:  	thread_id:63234	last_v:0	v:2

changed:	thread_id:63234	last_v:2	v:2



sleeped:	thread_id:63234	last_v:2	v:2

start:  	thread_id:63232	last_v:0	v:1

changed:	thread_id:63232	last_v:1	v:1

sleeped:	thread_id:63232	last_v:1	v:1

start:  	thread_id:63236	last_v:0	v:3

changed:	thread_id:63236	last_v:3	v:3

sleeped:	thread_id:63236	last_v:3	v:3

start:  	thread_id:63234	last_v:0	v:4

changed:	thread_id:63234	last_v:4	v:4

sleeped:	thread_id:63234	last_v:4	v:4

start:  	thread_id:63232	last_v:0	v:5

changed:	thread_id:63232	last_v:5	v:5

sleeped:	thread_id:63232	last_v:5	v:5

start:  	thread_id:63236	last_v:0	v:6

changed:	thread_id:63236	last_v:6	v:6

sleeped:	thread_id:63236	last_v:6	v:6



[1, 4, 9, 16, 25, 36]

In [21]:
import threading

def my_function():
    print("Thread ID:", threading.get_ident())

# Create and start a new thread
my_thread = threading.Thread(target=my_function)
my_thread.start()

Thread ID: 140204492969728


In [2]:
import concurrent.futures

# Your function that will be executed in parallel
def do(x):
    return x * 2

# List of arguments for the function
args_list = [1, 2, 3, 4, 5]

with concurrent.futures.ProcessPoolExecutor() as executor:
    # map() applies the do() function to each item in args_list in parallel
    futures = [executor.submit(do, x) for x in args_list]

# Wait for all the futures to complete
concurrent.futures.wait(futures)

# Collect the results
results = [future.result() for future in futures]
print(results)  # Output: [2, 4, 6, 8, 10]


[2, 4, 6, 8, 10]


In [2]:
from src import testing_report
import src.strategy.run_config as cfg
from src.strategy.run_report import  absRunReportFactory
from src.strategy.run_report.run_report import RunReport
from src.testing_report import TestingReport 
from datetime import datetime
from src.strategy.deal import Deal

si = cfg.StrategyId("test", "1.0")

# test set 1
sc_set1 = cfg.StrategyConfig({"P1": 1,"P2": 2})
rc1 = cfg.RunConfig(
    cfg.MarketConfig([cfg.StockConfig("S1", cfg.TimeFrame.D)],cfg.TimeFrame.D, cfg.date(2020, 1, 1), cfg.date(2021, 1, 1)),
    sc_set1)
cap1 = {
  datetime(2020,1,1):100.0,
  datetime(2020,1,2):102
}
deal_list1 = [
  Deal(datetime(2020,1,1),100,datetime(2020,1,10),120,1,100),
  Deal(datetime(2020,1,10),120,None,90,-1,120)
]
rr = RunReport(si, rc1, cap1,deal_list1)
tr = TestingReport([rr])


In [8]:
import src.strategy.run_config as rcfg
import pandas as pd

level_cut = 3
df_rec = []
for rr in tr.run_report_list:
    rr_dict = rr.to_dict()
    del rr_dict["abs_cap_log"]
    del rr_dict["deal_list"]
    flat_dict = TestingReport.flatten_dict(rr_dict)
    
    df_dict = {}
    for k,v in flat_dict.items():
        if len(k) == 2 and k[0]=="strategy_id":
            df_dict[("startegy", k[0],k[1])] = v
        else:
            df_dict[k]=v
    
    cutted_flat_dict = TestingReport.cut_level(df_dict, level_cut)
    df_rec.append(cutted_flat_dict)



{('startegy', 'strategy_id', 'name'): 'test',
 ('startegy', 'strategy_id', 'v'): '1.0',
 ('run_config', 'market_cfg', 'stocks', '0', 'ticker'): 'S1',
 ('run_config', 'market_cfg', 'stocks', '0', 'timeframe'): 'D',
 ('run_config', 'market_cfg', 'step_timeframe'): 'D',
 ('run_config', 'market_cfg', 'from_date'): datetime.date(2020, 1, 1),
 ('run_config', 'market_cfg', 'untill_date'): datetime.date(2021, 1, 1),
 ('run_config', 'strategy_cfg', 'P1'): 1,
 ('run_config', 'strategy_cfg', 'P2'): 2,
 ('metric', 'capital', 'yield'): 0.020000000000000018,
 ('metric', 'capital', 'yield/year'): 0.019944813843109976,
 ('metric', 'capital', 'max_yield'): 0.020000000000000018,
 ('metric', 'capital', 'max_fall'): 0,
 ('metric', 'deals', 'deal_count'): 2,
 ('metric', 'deals', 'success_deal_count'): 2,
 ('metric', 'deals', 'fail_deal_count'): 0,
 ('metric', 'deals', 'avg_net_profit'): 0.225,
 ('metric', 'deals', 'avg_net_income'): 0.225,
 ('metric', 'deals', 'avg_net_loss'): 0,
 ('metric', 'deals', 'PROM

In [9]:
cutted_flat_dict 

{('strategy_id', 'name'): 'test',
 ('strategy_id', 'v'): '1.0',
 ('run_config', 'market_cfg', 'stocks_0_ticker'): 'S1',
 ('run_config', 'market_cfg', 'stocks_0_timeframe'): 'D',
 ('run_config', 'market_cfg', 'step_timeframe'): 'D',
 ('run_config', 'market_cfg', 'from_date'): datetime.date(2020, 1, 1),
 ('run_config', 'market_cfg', 'untill_date'): datetime.date(2021, 1, 1),
 ('run_config', 'strategy_cfg', 'P1'): 1,
 ('run_config', 'strategy_cfg', 'P2'): 2,
 ('metric', 'capital', 'yield'): 0.020000000000000018,
 ('metric', 'capital', 'yield/year'): 0.019944813843109976,
 ('metric', 'capital', 'max_yield'): 0.020000000000000018,
 ('metric', 'capital', 'max_fall'): 0,
 ('metric', 'deals', 'deal_count'): 2,
 ('metric', 'deals', 'success_deal_count'): 2,
 ('metric', 'deals', 'fail_deal_count'): 0,
 ('metric', 'deals', 'avg_net_profit'): 0.225,
 ('metric', 'deals', 'avg_net_income'): 0.225,
 ('metric', 'deals', 'avg_net_loss'): 0,
 ('metric', 'deals', 'PROM'): 0.2928932188134524}

In [10]:
df = pd.DataFrame.from_records(df_rec)
df

,"(strategy_id, name)","(strategy_id, v)","(run_config, market_cfg, stocks_0_ticker)","(run_config, market_cfg, stocks_0_timeframe)","(run_config, market_cfg, step_timeframe)","(run_config, market_cfg, from_date)","(run_config, market_cfg, untill_date)","(run_config, strategy_cfg, P1)","(run_config, strategy_cfg, P2)","(metric, capital, yield)","(metric, capital, yield/year)","(metric, capital, max_yield)","(metric, capital, max_fall)","(metric, deals, deal_count)","(metric, deals, success_deal_count)","(metric, deals, fail_deal_count)","(metric, deals, avg_net_profit)","(metric, deals, avg_net_income)","(metric, deals, avg_net_loss)","(metric, deals, PROM)"
0,test,1.0,S1,D,D,2020-01-01,2021-01-01,1,2,0.02,0.019945,0.02,0,2,2,0,0.225,0.225,0,0.292893


In [ ]:
df = pd.DataFrame.from_records(df_rec)
df.columns = pd.MultiIndex.from_tuples(df.columns)
# drop const values
df.drop([RunReport.STRATEGY_ID_F],axis=1,inplace=True)
df

In [6]:
# join ticker cols
ticker_cols = [(RunReport.RUN_CONFIG_F, rcfg.RunConfig.MARKET_CFG_F, m) for m in  ["stocks_0_ticker",	"stocks_0_timeframe", rcfg.MarketConfig.STEP_TF_F]]
ticker_cols

[('run_config', 'market_cfg', 'stocks_0_ticker'),
 ('run_config', 'market_cfg', 'stocks_0_timeframe'),
 ('run_config', 'market_cfg', 'step_timeframe')]

In [4]:
df[(RunReport.RUN_CONFIG_F, rcfg.RunConfig.MARKET_CFG_F,"full_ticker")] = df[[(RunReport.RUN_CONFIG_F, rcfg.RunConfig.MARKET_CFG_F, m) for m in  ["stocks_0_ticker",	"stocks_0_timeframe",rcfg.MarketConfig.STEP_TF_F]]].apply(lambda row: ':'.join(map(str, row)), axis=1)


ValueError: Item must have length equal to number of levels.

In [ ]:
df.drop(ticker_cols, axis=1,inplace=True)
# set index
df.set_index([c for c in df.columns if c[0] ==RunReport.RUN_CONFIG_F], inplace=True)
# drop unused level
df.index.names = [(n[1],n[2]) for n in  df.index.names] 
df.columns = df.columns.droplevel(0)
df = df.reorder_levels([df.index.names[-1],*df.index.names[:-1]])
df